# Imports
* Dataset used - MNIST (with one-hot-encoding)
* LOG_DIR = "tmp/"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
LOG_DIR = "tmp/"

## Input and Ground-truth placeholders  
* One image is a 784 length vector with 0's and 1's  
* Ground-truth is a label from 0-9  
.  
* Y  = ground-truth  
* Y-underscore = predicted-output

In [ ]:
# Input 28-by-28 pixels images of GRAYSCALE
X = tf.placeholder(tf.float32, [None, 784])
# Output in 'one-hot-encoding', 10 classes
Y_ = tf.placeholder(tf.float32, [None, 10])

# Architecture
* **Input-layer**[784 neurons] >> (W, b) >> **Output-layer**[10 neurons]
* **softmax** activation function used at output layer  

### Dimensions
* W[784, 10]  
* b[10]  
 
### Cost function  
* cost = L1 norm 
  
### Accuracy function  
* Accuracy = (correct)/(correct + incorrect) %  

In [ ]:
# model
W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.75))
tf.summary.histogram("weights", W)
b = tf.Variable(tf.truncated_normal([10], stddev=0.5))
tf.summary.histogram("biases", b)
# model-output
Y = (tf.matmul(X, W) + b)

# variable-initializer
init = tf.global_variables_initializer()

# cost
cross_entropy = tf.reduce_sum(tf.abs(Y - Y_))
tf.summary.scalar("cross_entropy", cross_entropy)

# accuracy
is_correct = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
tf.summary.scalar("accuracy", accuracy)

## Hyperparameters and train_step
* learning_rate
* number_of_epochs
* batch_size

In [ ]:
# hyper parameters
learning_rate = 0.003
n_epochs = 1000
batch_size = 100

tf.summary.scalar("learning_rate", learning_rate)
tf.summary.scalar("number_of_epochs", n_epochs)
tf.summary.scalar("mini_batch_size", batch_size)


# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

# Training
* Mini-batch method used
* train & test results stored separately

In [ ]:
# training
sess = tf.InteractiveSession()
sess.run(init)
summaries = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(LOG_DIR + "train", graph=tf.get_default_graph())
test_writer = tf.summary.FileWriter(LOG_DIR + "test", graph=tf.get_default_graph())

for i in range(n_epochs):
    # feed train-data
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    train_data = {X: batch_X, Y_: batch_Y}
    
    summ, _ = sess.run([summaries, train_step], feed_dict=train_data)
    train_writer.add_summary(summ, global_step=i)
    
    # feed test-data
    test_data = {X: mnist.test.images, Y_: mnist.test.labels}
    
    summ = sess.run(summaries, feed_dict=test_data)
    test_writer.add_summary(summ, global_step=i)
    
    # print
    if i%4 == 0:
        print ("Iteration: ", i, " complete")